In [2]:
import torch
from torch import nn
from torch.nn import functional as F

## The Cross-Correlation Operation

In [30]:
def corr2d(X, K):
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1]- w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    return Y
            
X = torch.arange(9).reshape(3, 3)
K = torch.arange(4).reshape(2, 2)
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

In [31]:
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [32]:
K = torch.tensor([-1, 1]).reshape(-1, 2)
Y = corr2d(X, K)
Y

tensor([[ 0., -1.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  1.,  0.]])

In [33]:
corr2d(X.T, K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

## Convolutional Layers

In [34]:
class conv2d(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))
        
    def forward(self, X):
        return corr2d(X, self.weight) + self.bias
    
net = conv2d((2, 2))

In [44]:
conv2d = nn.Conv2d(1, 1, kernel_size=(1, 2), bias=False)
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))
lr = 3e-2

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y - Y_hat) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    conv2d.weight.data -= lr * conv2d.weight.grad
    if (i + 1) % 2 == 0:
        print(f'epoch {i + 1}, loss {l.sum():.3f}')

epoch 2, loss 10.522
epoch 4, loss 2.673
epoch 6, loss 0.820
epoch 8, loss 0.290
epoch 10, loss 0.111


In [45]:
conv2d.weight

Parameter containing:
tensor([[[[-0.9529,  1.0200]]]], requires_grad=True)

## Padding and Stride 

In [48]:
def comp_conv2d(conv2d, X):
    X = X.reshape((1, 1) + X.shape)
    Y = conv2d(X)
    return Y.reshape(Y.shape[2:])

conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1)
X = torch.rand(8, 8)
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

In [50]:
conv2d = nn.Conv2d(1, 1, kernel_size=(5, 3), padding=(2, 1))
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

In [51]:
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1, stride=2)
comp_conv2d(conv2d, X).shape

torch.Size([4, 4])

## Multiple Input Channels

In [99]:
def corr2d_multi_in(X, K):
    return sum(corr2d(x, k) for x, k in zip(X, K))

X = torch.stack((torch.arange(9).reshape(3, 3), torch.arange(1, 10).reshape(3, 3))).reshape(2, 3, 3)
K = torch.stack((torch.arange(4).reshape(2, 2), torch.arange(1, 5).reshape(2, 2))).reshape(2, 2, 2)
corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

In [101]:
def corr2d_multi_out(X, K):
    return torch.stack([corr2d_multi_in(X, k) for k in K])

K = torch.stack((K, K + 1, K + 2), 0)
corr2d_multi_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

In [115]:
def corr2d_multi_in_out_1x1(X, K):
    ci, h, w = X.shape
    co = K.shape[0]
    X = X.reshape((ci, h * w))
    K = K.reshape((co, ci))
    Y = torch.matmul(K, X)
    return Y.reshape(co, h, w)

X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(2, 3, (2, 3, 1, 1))
Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_out(X, K)
Y1
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6

## Pooling

In [140]:
def pool2d(X, pool_size, mode="max"):
    p_h, p_w = pool_size
    Y = torch.zeros((X.shape[0] - p_h + 1, X.shape[1] - p_w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode == "max":
                Y[i, j] = (X[i:i + p_h, j:j + p_w]).max()
            elif mode == "avg":
                Y[i, j] = (X[i:i + p_h, j:j + p_w]).mean()
    return Y


X = torch.arange(9, dtype=torch.float).reshape(3, 3)
pool2d(X, (2, 2))

tensor([[4., 5.],
        [7., 8.]])

In [141]:
pool2d(X, (2, 2), 'avg')

tensor([[2., 3.],
        [5., 6.]])

In [145]:
X = torch.arange(16, dtype=torch.float32).reshape(1, 4, 4)
pool_2d = nn.MaxPool2d(3)
pool_2d(X)

tensor([[[10.]]])

In [146]:
pool_2d = nn.MaxPool2d(3, padding=1, stride=2)
pool_2d(X)

tensor([[[ 5.,  7.],
         [13., 15.]]])

In [147]:
X = torch.stack([X, X + 1], 1)
X.shape

torch.Size([1, 2, 4, 4])

In [150]:
pool_2d = nn.MaxPool2d(3, padding=(0, 1), stride=(2, 2))
pool_2d(X)

tensor([[[[ 9., 11.]],

         [[10., 12.]]]])